# Scaffold-based reinforcement learning and molecule generation

In some cases you might have idea of a scaffold or fragments that your new molecules should contain. In this case, it usefull to do the [reinforcement learning](rl_optimization.ipynb) and the [molecule generation](generation.ipynb) with preselected fragments (single fragment or combination of multiple fragments).

In this tutorial, we show an example with a single pyrazine and a combination of a pyrazine and a thiophene for the smiles- and graph-based transformer.

To understand this tutorial and to have all necessary files, we expect the user to be familiar with the basic tutorials on [data preprocessing](../datasets.ipynb), [reinforcement learning](rl_optimization.ipynb) and the [molecule generation](generation.ipynb).

In [15]:
%load_ext autoreload
%autoreload 2

In [41]:
import sys
sys.path.append('/zfsdata/data/sohvi/DrugEx/')

sys.path.append('..')
from utils import smilesToGrid

frags = ['c1cnccn1', 'c1cnccn1.c1ccsc1' ]  

smilesToGrid(frags)


# Building the environment

In this example, we build both the graph- and smiles-based models with the same vocabularies, pretrained and finetuned generators and QSAR models as in the [general RL example](../rl_optimization.ipynb).

First we build the environment which is identique for both models,

In [53]:
from drugex.training.scorers.properties import Property
from drugex.training.scorers.modifiers import ClippedScore
from drugex.training.environment import DrugExEnvironment
from drugex.training.rewards import ParetoCrowdingDistance
from drugex.training.scorers.predictors import Predictor
from drugex.training.interfaces import Scorer

class ModelScorer(Scorer):
    
    def __init__(self, model, prefix):
        super().__init__()
        self.model = model
        self.prefix = prefix
    
    def getScores(self, mols, frags=None):
        X = Predictor.calculateDescriptors(mols)
        return self.model.predict(X)
    
    def getKey(self):
        return f"{self.prefix}_{type(self.model)}"

qed = Property("QED", modifier=ClippedScore(lower_x=0, upper_x=1.0))
sascore = Property("SA", modifier=ClippedScore(lower_x=4.5, upper_x=0))
scorers = [qed, sascore]
thresholds = [0.5, 0.5]

# scorer_a1 = joblib.load('data/models/reinforced/graph/scorer_a1.pkg') #pickle.load(open('data/models/reinforced/graph/scorer_a1.pkg', 'rb'))
# scorer_a3 = joblib.load('data/models/reinforced/graph/scorer_a3.pkg') #pickle.load(open('data/models/reinforced/graph/scorer_a3.pkg', 'rb'))
# scorers = [scorer_a1, scorer_a3, qed, sascore]
# thresholds = [0.99, 0.99, 0.0, 0.0]

environment = DrugExEnvironment(scorers, thresholds, reward_scheme=ParetoCrowdingDistance())

# Graph-based Transformer
## Data Preprocessing

We use the same encoder as in [Preparing Data for the Graph-Based Transformer](../datasets.ipynb) to create molecules from the fragments and encode fragment-molecule pairs, with a small modifications:
1. Instead of using a `fragmenter` we create dummy molecules from the fragments with `dummyMolsFromFragments` 
2. Set `splitter` to `None`, `n_proc` and `chunk_size` to 1 

In [44]:
import os
from drugex.data.datasets import GraphFragDataSet
from drugex.molecules.converters.dummy_molecules import dummyMolsFromFragments
from drugex.data.fragments import FragmentCorpusEncoder, GraphFragmentEncoder
from drugex.data.corpus.vocabulary import VocGraph

fragmenter = dummyMolsFromFragments()
splitter = None

encoder = FragmentCorpusEncoder(
    fragmenter=fragmenter, 
    encoder=GraphFragmentEncoder(
        VocGraph(n_frags=4) 
    ),
    pairs_splitter=splitter, 
    n_proc=1,
    chunk_size=1
)

graph_input_folder = "data/sets/graph/"
if not os.path.exists(graph_input_folder):
    os.makedirs(graph_input_folder)
    
dataset = GraphFragDataSet(f"{graph_input_folder}/scaffold_graph.tsv", rewrite=True)

In [45]:
encoder.apply(list(frags), encodingCollectors=[dataset])

Creating fragment-molecule pairs (batch processing): 100%|██████████| 2/2 [00:00<00:00, 130.53it/s]
Encoding fragment-molecule pairs. (batch processing): 100%|██████████| 2/2 [00:00<00:00, 51.92it/s]


## Reinforcement learning

Then we can build the explorer composed of the agent, the prior and the enviroment.

In [54]:
from drugex.training.models.explorer import GraphExplorer
from drugex.training.models.transform import GraphModel
from drugex.data.corpus.vocabulary import VocGraph

GPUS = gpus=(1,)

vocabulary = VocGraph.fromFile('../data/models/finetuned/graph/ligand_finetuned.vocab')
finetuned = GraphModel(voc_trg=vocabulary, use_gpus=GPUS)
finetuned.loadStatesFromFile('../data/models/finetuned/graph/chembl_ligand.pkg')
pretrained = GraphModel(voc_trg=vocabulary, use_gpus=GPUS)
pretrained.loadStatesFromFile('../jupyter/models/pretrained/graph/chembl27/chembl27_graph.pkg')

explorer = GraphExplorer(agent=pretrained, env=environment, mutate=finetuned, epsilon=0.1, use_gpus=GPUS)

But used only the selected scaffolds as input fragments for training and validation. As the initial set only contains two inputs, they are sampled 100 times to create the training set and 100*0.2=20 to create the test set.

In [55]:
from drugex.data.datasets import GraphFragDataSet

data_path = 'data/sets/graph/scaffold_graph.tsv'
train_loader = GraphFragDataSet(data_path).asDataLoader(batch_size=1024, n_samples=100)
test_loader = GraphFragDataSet(data_path).asDataLoader(batch_size=1024, n_samples=100, n_samples_ratio=0.2)

After that we can finally start the training loop:

In [56]:
from drugex.training.monitors import FileMonitor

monitor = FileMonitor("data/models/reinforced/graph/scaffold_rl", verbose=True) 
explorer.fit(train_loader, test_loader, monitor=monitor, epochs=3)

100%|██████████| 3/3 [00:11<00:00,  3.92s/it]


We look that all created molecules include either a pyrazine or a pyrazine and a thiophene.

In [61]:
import pandas as pd 

df_smiles = pd.read_csv('data/models/reinforced/graph/scaffold_rl_smiles.tsv', sep='\t')
smilesToGrid(df_smiles.Smiles.tolist())

## *de novo* generation

Once we have the optimized model (not the case in tutorial as for speed is set to 3 instead of 1000), it can be used to sample *novel*  mocules.

In [72]:
reinforced = GraphModel(voc_trg=VocGraph.fromFile('data/sets/graph/scaffold_graph.tsv.vocab'), use_gpus=GPUS)
reinforced.loadStatesFromFile('data/models/reinforced/graph/scaffold_rl.pkg')

gen_loader = GraphFragDataSet('data/sets/graph/scaffold_graph.tsv').asDataLoader(batch_size=1024)

Novel molecules can be generated with the modified scores (to evaluate to ratio of desired molecules by model),

In [75]:
frags_, smiles, scores = reinforced.evaluate(gen_loader, repeat=100, method=environment)
scores['SMILES'], scores['Frags'] = smiles, frags_

In [78]:
scores.head()

,QED,SA,DESIRE,VALID,SMILES,Frags
0,0.785529,0.566614,1,1,CCN(Cc1ccccc1)Cc1cnccn1,c1cnccn1
1,0.502609,0.367376,0,1,S=C(NN=Cc1cnccn1)Nc1cccs1,c1ccsc1.c1cnccn1
2,0.195188,0.333929,0,1,COc1ccccc1-c1cc(-c2nc(N3CCN(C)CC3)cnc2-c2ccc(F...,c1ccsc1.c1cnccn1
3,0.756322,0.438684,0,1,N#Cc1ccc(Nc2ncc(-c3cnccn3)[nH]2)cc1,c1cnccn1
4,0.927767,0.117580,0,1,CCCN1C2CCCCC1CC(NC(=O)c1cnccn1)CC2,c1cnccn1


In [77]:
desired_smiles = scores[scores.DESIRE ==1].SMILES.tolist()
smilesToGrid(desired_smiles)

or without applying the modifiers to better evaluate the predicted properties.

In [80]:
for scorer in environment.scorers:
    scorer.modifier=None
frags_, smiles, scores = reinforced.evaluate(gen_loader, repeat=100, method=environment)
scores['SMILES'], scores['Frags'] = smiles, frags_
scores = scores.drop('DESIRE', axis=1) # without the modifiers the desirability is meaningless
scores.head()

,QED,SA,VALID,SMILES,Frags
0,0.919373,4.258730,1,Fc1cnc(NC2C3COC3C2c2ccccn2)cn1,c1cnccn1
1,0.934854,3.696073,1,CC(=O)N(C)c1cnc(N2CCC(O)C2c2cccs2)cn1,c1ccsc1.c1cnccn1
2,0.673741,3.584340,1,CCCCNC(=O)c1cc(CC2C(NC(=O)c3cnccn3)CN2C)cs1,c1ccsc1.c1cnccn1
3,0.466039,3.125878,1,CC(=O)C(=CNc1cncc(F)n1)C(=O)NCc1cc(C)on1,c1cnccn1
4,0.224607,2.857892,1,Cc1nc(N2CCN(c3ccc(C=Cc4ccc(C#N)cc4)cc3)CC2)cc2...,c1cnccn1


# SMILES-based Transformer
## Data prepocessing

We use the same encoder as in [Preparing Data for the SMILES-Based Transformer](../datasets.ipynb) to create molecules from the fragments and encode fragment-molecule pairs, with a small modifications:
1. Instead of using a `fragmenter` we create dummy molecules from the fragments with `dummyMolsFromFragments` 
2. Set `splitter` to `None`, `min_len` to 2, `n_proc` and `chunk_size` to 1 

In [59]:
import os
from drugex.data.datasets import SmilesFragDataSet
from drugex.molecules.converters.dummy_molecules import dummyMolsFromFragments
from drugex.data.fragments import FragmentCorpusEncoder, SequenceFragmentEncoder
from drugex.data.corpus.vocabulary import VocSmiles

fragmenter = dummyMolsFromFragments()
splitter = None

encoder = FragmentCorpusEncoder(
    fragmenter=fragmenter, 
    encoder=SequenceFragmentEncoder(
        VocSmiles(min_len=2) 
    ),
    pairs_splitter=splitter, 
    n_proc=1,
    chunk_size=1
)

smiles_input_folder = "data/sets/smiles/"
if not os.path.exists(smiles_input_folder):
    os.makedirs(smiles_input_folder)
    
dataset = SmilesFragDataSet(f"{smiles_input_folder}/scaffold_smi.tsv", rewrite=True)

Initialized empty dataset. The data set file does not exist (yet): data/sets/smiles//scaffold_smi.tsv. You can add data by calling this instance with the appropriate parameters.


In [60]:
encoder.apply(list(frags), encodingCollectors=[dataset])

Creating fragment-molecule pairs (batch processing): 100%|██████████| 2/2 [00:00<00:00, 36.00it/s]
Encoding fragment-molecule pairs. (batch processing): 100%|██████████| 2/2 [00:00<00:00, 39.80it/s]


## Reinforcement learning

!!! Does not work yet as we do not have pretrained/finetuned SMILES-based transformer models available !!!!

and the explorer composed of the agent, the prior and the enviroment.

In [58]:
from drugex.training.models.explorer import SmilesExplorer
from drugex.training.models.transform import GPT2Model
from drugex.data.corpus.vocabulary import VocSmiles

GPUS = gpus=(0,1)

vocabulary = VocSmiles.fromFile('../data/models/finetuned/smiles/ligand_finetuned.vocab')
finetuned = GPT2Model(voc_trg=vocabulary, use_gpus=GPUS)
finetuned.loadStatesFromFile('../data/models/finetuned/smiles/chembl_ligand.pkg')
pretrained = GPT2Model(voc_trg=vocabulary, use_gpus=GPUS)
pretrained.loadStatesFromFile('../jupyter/models/pretrained/smiles/chembl27/chembl27_graph.pkg')

explorer = SmilesExplorer(agent=pretrained, env=environment, mutate=finetuned, epsilon=0.1, use_gpus=GPUS)

FileNotFoundError: [Errno 2] No such file or directory: '../data/models/finetuned/smiles/ligand_finetuned.vocab'

But used only the selected scaffolds as input fragments for training and validation. As the initial set only contains two inputs, they are sampled 100 times to create the training set and 100*0.2=20 to create the test set.

In [30]:
from drugex.data.datasets import SmilesFragDataSet

data_path = 'data/sets/smiles/scaffold_smi.tsv'
train_loader = SmilesFragDataSet(data_path).asDataLoader(batch_size=1024, n_samples=100)
test_loader = SmilesFragDataSet(data_path).asDataLoader(batch_size=1024, n_samples=100, n_samples_ratio=0.2)

After that we can finally start the training loop:

In [ ]:
from drugex.training.monitors import FileMonitor

monitor = FileMonitor("data/models/reinforced/smiles/scaffold_rl", verbose=True) 
explorer.fit(train_loader, test_loader, monitor=monitor, epochs=3)

100%|██████████| 3/3 [00:14<00:00,  4.75s/it]
